In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

from tqdm import tqdm

if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('GPU')

else:
  device = torch.device('cpu')
  print('CPU')



GPU


In [ ]:
from torch.utils.data import Subset

# Define the root directory of your dataset and your desired image size
data_root = '/content/drive/MyDrive/traindata'
desired_size = (300, 300)  # Adjust to your desired size

# Create a dataset with the same transformations you are using
transform = transforms.Compose([
    transforms.Resize(desired_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Resize images to the desired size

])

custom_dataset = ImageFolder(root=data_root, transform=transform)



In [ ]:
batch_size = 64


train_size = int(0.8 * len(custom_dataset))
test_size = len(custom_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('cherry','strawberry','tomato')

In [ ]:
class_to_idx = custom_dataset.class_to_idx

# Invert the mapping to get class labels to counts
class_counts = {class_label: 0 for class_label in class_to_idx}

# Count the occurrences of each class label
for _, label in train_dataset:
    class_counts[custom_dataset.classes[label]] += 1

# Print the counts
for class_label, count in class_counts.items():
    print(f'Class {class_label}: {count} samples')

Class cherry: 1183 samples
Class strawberry: 1207 samples
Class tomato: 1210 samples


Baseline Model

In [ ]:
#split the data into test & training
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3*300*300, 128) #input_size = 3 * 300 * 300
        self.fc2 = nn.Linear(128,3)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Update the view dimension

        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

net = Net()

In [ ]:
import torch.optim as optim

net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)



for epoch in tqdm(range(5)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

100%|██████████| 5/5 [06:57<00:00, 83.46s/it]

Finished Training


In [ ]:
net.eval()


correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)


        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 3000 test images: {100 * correct // total} %')

Accuracy of the network on the 3000 test images: 51 %


Basic Convilutional Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 72 * 72, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)



for epoch in tqdm(range(5)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

100%|██████████| 5/5 [01:03<00:00, 12.77s/it]

Finished Training


In [ ]:
net.eval()


correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)


        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 3000 test images: {100 * correct // total} %')

Accuracy of the network on the 3000 test images: 55 %


Cross-Validating Optimisers

In [ ]:
import torch.optim as optim
import time
from sklearn.model_selection import KFold

folds = 3
classes = ('cherry', 'strawberry', 'tomato')

# Pre-split your dataset into a training set and a test set
train_size = int(0.8 * len(custom_dataset))
test_size = len(custom_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size])

batch_size = 64

# Create a KFold object to handle the splits for the training set
kf = KFold(n_splits=folds, shuffle=True)

for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset), 1):
    print(f"Fold {fold}:")

    # Create a validation dataset for this fold
    train_split_dataset = torch.utils.data.Subset(train_dataset, train_indices)
    val_dataset = torch.utils.data.Subset(train_dataset, val_indices)

    trainloader = torch.utils.data.DataLoader(train_split_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    net = Net()
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    for epoch in tqdm(range(5)):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    print('Finished Training')

    net.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for data in valloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # calculate outputs by running images through the network
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the validation set: {accuracy:.2f}%')

    # You can save the model with the best validation performance here (if needed)

# Test the model on the separate test set
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

net.eval()

correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # calculate outputs by running images through the network
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the test set: {accuracy:.2f}%')

print("K-fold cross-validation and testing on the test set completed.")


Fold 1:


100%|██████████| 5/5 [00:55<00:00, 11.00s/it]

Finished Training


Accuracy of the network on the validation set: 52.00%
Fold 2:


100%|██████████| 5/5 [00:50<00:00, 10.18s/it]

Finished Training


Accuracy of the network on the validation set: 48.50%
Fold 3:


100%|██████████| 5/5 [00:53<00:00, 10.63s/it]

Finished Training


Accuracy of the network on the validation set: 43.33%
Accuracy of the network on the test set: 42.78%
K-fold cross-validation and testing on the test set completed.


Adam

In [ ]:
import torch.optim as optim
import time
from sklearn.model_selection import KFold

folds = 3
classes = ('cherry', 'strawberry', 'tomato')

# Pre-split your dataset into a training set and a test set
train_size = int(0.8 * len(custom_dataset))
test_size = len(custom_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size])

batch_size = 64

# Create a KFold object to handle the splits for the training set
kf = KFold(n_splits=folds, shuffle=True)

for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset), 1):
    print(f"Fold {fold}:")

    # Create a validation dataset for this fold
    train_split_dataset = torch.utils.data.Subset(train_dataset, train_indices)
    val_dataset = torch.utils.data.Subset(train_dataset, val_indices)

    trainloader = torch.utils.data.DataLoader(train_split_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    net = Net()
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)  # Use the Adam optimizer

    for epoch in tqdm(range(5)):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    print('Finished Training')

    net.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for data in valloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # calculate outputs by running images through the network
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the validation set: {accuracy:.2f}%')

    # You can save the model with the best validation performance here (if needed)

# Test the model on the separate test set
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

net.eval()

correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # calculate outputs by running images through the network
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the test set: {accuracy:.2f}%')

print("K-fold cross-validation and testing on the test set completed.")

Fold 1:


100%|██████████| 5/5 [00:46<00:00,  9.30s/it]

Finished Training


Accuracy of the network on the validation set: 54.58%
Fold 2:


100%|██████████| 5/5 [00:43<00:00,  8.77s/it]

Finished Training


Accuracy of the network on the validation set: 52.42%
Fold 3:


100%|██████████| 5/5 [00:44<00:00,  8.91s/it]

Finished Training


Accuracy of the network on the validation set: 50.00%
Accuracy of the network on the test set: 50.22%
K-fold cross-validation and testing on the test set completed.


In [ ]:
(54.58+52.42+50)/3

52.333333333333336

RMSProp

In [ ]:
import torch.optim as optim
import time
from sklearn.model_selection import KFold

folds = 3
classes = ('cherry', 'strawberry', 'tomato')

# Pre-split your dataset into a training set and a test set
train_size = int(0.8 * len(custom_dataset))
test_size = len(custom_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size])

batch_size = 64

# Create a KFold object to handle the splits for the training set
kf = KFold(n_splits=folds, shuffle=True)

for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset), 1):
    print(f"Fold {fold}:")

    # Create a validation dataset for this fold
    train_split_dataset = torch.utils.data.Subset(train_dataset, train_indices)
    val_dataset = torch.utils.data.Subset(train_dataset, val_indices)

    trainloader = torch.utils.data.DataLoader(train_split_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    net = Net()
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.RMSprop(net.parameters(), lr=0.001)  # Use the RMSprop optimizer

    for epoch in tqdm(range(5)):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    print('Finished Training')

    net.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for data in valloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # calculate outputs by running images through the network
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the validation set: {accuracy:.2f}%')

    # You can save the model with the best validation performance here (if needed)

# Test the model on the separate test set
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

net.eval()

correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # calculate outputs by running images through the network
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the test set: {accuracy:.2f}%')

print("K-fold cross-validation and testing on the test set completed.")

Fold 1:


100%|██████████| 5/5 [00:47<00:00,  9.57s/it]

Finished Training


Accuracy of the network on the validation set: 41.17%
Fold 2:


100%|██████████| 5/5 [00:45<00:00,  9.05s/it]

Finished Training


Accuracy of the network on the validation set: 48.75%
Fold 3:


100%|██████████| 5/5 [00:43<00:00,  8.70s/it]

Finished Training


Accuracy of the network on the validation set: 46.83%
Accuracy of the network on the test set: 48.00%
K-fold cross-validation and testing on the test set completed.


In [ ]:
import torch.optim as optim
net = Net()

net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)  # Use the Adam optimizer



for epoch in tqdm(range(5)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

100%|██████████| 5/5 [00:55<00:00, 11.04s/it]

Finished Training


In [ ]:
net.eval()


correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)


        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 3000 test images: {100 * correct // total} %')

Accuracy of the network on the 3000 test images: 62 %


In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = net(inputs)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: cherry is 50.3 %
Accuracy for class: strawberry is 49.7 %
Accuracy for class: tomato is 88.8 %


MiniBatches

In [ ]:
import pandas as pd
import torch.optim as optim
from sklearn.model_selection import KFold
import time

# Initialize an empty DataFrame to store the results
results = pd.DataFrame(columns=["Batch Size", "Fold 1", "Fold 2", "Fold 3", "Average", "Time"])

# Define the list of batch sizes to test
batch_sizes = [32, 64, 128]

folds = 3
classes = ('cherry', 'strawberry', 'tomato')

# Create a KFold object to handle the splits for the training set
kf = KFold(n_splits=folds, shuffle=True)

for batch_size in batch_sizes:
    batch_scores = []
    batch_time = []
    for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset), 1):
        print(f"Testing Batch Size {batch_size}, Fold {fold}:")

        train_split_dataset = torch.utils.data.Subset(train_dataset, train_indices)
        val_dataset = torch.utils.data.Subset(train_dataset, val_indices)

        trainloader = torch.utils.data.DataLoader(train_split_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
        valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

        net = Net()
        net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(net.parameters(), lr=0.001)

        start_time = time.time()
        for epoch in tqdm(range(5)):
            running_loss = 0.0
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        # Test the model on the separate test set
        net.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for data in valloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy_val = 100 * correct / total

        end_time = time.time()
        elapsed_time = end_time - start_time

        # Append fold score to batch_scores
        batch_scores.append(accuracy_val)
        # Append elapsed time to batch_time
        batch_time.append(elapsed_time)

    # Calculate the average score for the current batch size
    average_score = sum(batch_scores) / len(batch_scores)

    # Add the results to the DataFrame
    results = results.append({
        "Batch Size": batch_size,
        "Fold 1": f"{batch_scores[0]:.2f}%",
        "Fold 2": f"{batch_scores[1]:.2f}%",
        "Fold 3": f"{batch_scores[2]:.2f}%",
        "Average": f"{average_score:.2f}%",
        "Time": sum(batch_time)
    }, ignore_index=True)

# Display the DataFrame
print(results)

# Calculate and display the average scores for each batch size
average_results = results.groupby("Batch Size").mean()
average_results = average_results.drop(columns="Time")
print("\nAverage Results:")
print(average_results)


Testing Batch Size 32, Fold 1:


100%|██████████| 5/5 [00:51<00:00, 10.39s/it]


Testing Batch Size 32, Fold 2:


100%|██████████| 5/5 [00:44<00:00,  8.92s/it]


Testing Batch Size 32, Fold 3:


...

100%|██████████| 5/5 [00:44<00:00,  8.96s/it]


Testing Batch Size 128, Fold 3:


100%|██████████| 5/5 [00:44<00:00,  8.92s/it]


  Batch Size  Fold 1  Fold 2  Fold 3 Average        Time
0         32  51.58%  53.75%  46.83%  50.72%  156.538980
1         64  53.25%  49.75%  55.75%  52.92%  149.654063
2        128  52.42%  41.33%  49.58%  47.78%  149.377690

Average Results:
Empty DataFrame
Columns: []
Index: [32, 64, 128]


<ipython-input-27-c03d4b7cd8eb>:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-27-c03d4b7cd8eb>:94: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_results = results.groupby("Batch Size").mean()


Learning Rate

In [ ]:
import pandas as pd
import torch.optim as optim
from sklearn.model_selection import KFold
import time

# Initialize an empty DataFrame to store the results
results = pd.DataFrame(columns=["Learning Rate", "Fold 1", "Fold 2", "Fold 3", "Average", "Time"])

# Define the list of learning rates to test
learning_rates = [0.001, 0.0015, 0.003, 0.005, 0.01]

batch_size = 64
folds = 3
classes = ('cherry', 'strawberry', 'tomato')

# Create a KFold object to handle the splits for the training set
kf = KFold(n_splits=folds, shuffle=True)

for learning_rate in learning_rates:
    batch_scores = []
    batch_time = []
    for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset), 1):
        print(f"Testing Learning Rate {learning_rate}, Batch Size {batch_size}, Fold {fold}:")

        train_split_dataset = torch.utils.data.Subset(train_dataset, train_indices)
        val_dataset = torch.utils.data.Subset(train_dataset, val_indices)

        trainloader = torch.utils.data.DataLoader(train_split_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
        valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

        net = Net()
        net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(net.parameters(), lr=learning_rate)

        start_time = time.time()
        for epoch in tqdm(range(5)):
            running_loss = 0.0
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        # Test the model on the separate test set
        net.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for data in valloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy_val = 100 * correct / total

        end_time = time.time()
        elapsed_time = end_time - start_time

        # Append fold score to batch_scores
        batch_scores.append(accuracy_val)
        # Append elapsed time to batch_time
        batch_time.append(elapsed_time)

    # Calculate the average score for the current learning rate
    average_score = sum(batch_scores) / len(batch_scores)

    # Add the results to the DataFrame
    results = results.append({
        "Learning Rate": learning_rate,
        "Fold 1": f"{batch_scores[0]:.2f}%",
        "Fold 2": f"{batch_scores[1]:.2f}%",
        "Fold 3": f"{batch_scores[2]:.2f}%",
        "Average": f"{average_score:.2f}%",
        "Time": sum(batch_time)
    }, ignore_index=True)

# Display the DataFrame
print(results)


Testing Learning Rate 0.001, Batch Size 64, Fold 1:


100%|██████████| 5/5 [00:48<00:00,  9.74s/it]


Testing Learning Rate 0.001, Batch Size 64, Fold 2:


100%|██████████| 5/5 [00:45<00:00,  9.16s/it]


Testing Learning Rate 0.001, Batch Size 64, Fold 3:


...

100%|██████████| 5/5 [00:44<00:00,  8.88s/it]


Testing Learning Rate 0.01, Batch Size 64, Fold 3:


100%|██████████| 5/5 [00:45<00:00,  9.17s/it]


   Learning Rate  Fold 1  Fold 2  Fold 3 Average        Time
0         0.0010  55.50%  50.08%  54.75%  53.44%  152.383169
1         0.0015  52.00%  49.17%  52.42%  51.19%  149.701398
2         0.0030  46.08%  42.25%  33.92%  40.75%  149.672995
3         0.0050  33.92%  31.92%  33.33%  33.06%  152.402562
4         0.0100  33.92%  31.83%  32.58%  32.78%  150.866724


<ipython-input-34-db3c35a62754>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({


Number of Convolutional Layers

In [ ]:
class Conv1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(131424, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(x.size(0), -1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class Conv2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 72 * 72, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class Conv3(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(36992, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class Conv4(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.conv4 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(14400, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))

        x = x.view(x.size(0), -1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class Conv5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.conv4 = nn.Conv2d(32, 64, 5)
        self.conv5 = nn.Conv2d(64, 128, 5)
        self.fc1 = nn.Linear(3200, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(x.size(0), -1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
import pandas as pd
import torch.optim as optim
from sklearn.model_selection import KFold
import time
import torch.nn as nn
import torch.nn.functional as F

import warnings

# To ignore all warnings (not recommended unless you know the risks)
warnings.filterwarnings("ignore")


# Initialize an empty DataFrame to store the results
results = pd.DataFrame(columns=["Model", "Fold 1", "Fold 2", "Fold 3", "Average", "Time"])

# Define the list of models to test
models = [Conv5]

learning_rate = 0.001
batch_size = 64
folds = 3
classes = ('cherry', 'strawberry', 'tomato')

# Create a KFold object to handle the splits for the training set
kf = KFold(n_splits=folds, shuffle=True)

for model_class in models:
    batch_scores = []
    batch_time = []
    for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset), 1):
        print(f"Testing Model {model_class.__name__}, Fold {fold}:")

        train_split_dataset = torch.utils.data.Subset(train_dataset, train_indices)
        val_dataset = torch.utils.data.Subset(train_dataset, val_indices)

        trainloader = torch.utils.data.DataLoader(train_split_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
        valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

        net = model_class()
        net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(net.parameters(), lr=learning_rate)

        start_time = time.time()
        for epoch in tqdm(range(5)):
            running_loss = 0.0
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        # Test the model on the separate test set
        net.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for data in valloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy_val = 100 * correct / total

        end_time = time.time()
        elapsed_time = end_time - start_time

        # Append fold score to batch_scores
        batch_scores.append(accuracy_val)
        # Append elapsed time to batch_time
        batch_time.append(elapsed_time)

    # Calculate the average score for the current model
    average_score = sum(batch_scores) / len(batch_scores)

    # Add the results to the DataFrame
    results = results.append({
        "Model": model_class.__name__,
        "Fold 1": f"{batch_scores[0]:.2f}%",
        "Fold 2": f"{batch_scores[1]:.2f}%",
        "Fold 3": f"{batch_scores[2]:.2f}%",
        "Average": f"{average_score:.2f}%",
        "Time": sum(batch_time)
    }, ignore_index=True)

# Display the DataFrame
print(results)


Testing Model Conv5, Fold 1:


100%|██████████| 5/5 [00:47<00:00,  9.42s/it]


Testing Model Conv5, Fold 2:


100%|██████████| 5/5 [00:46<00:00,  9.25s/it]


Testing Model Conv5, Fold 3:


100%|██████████| 5/5 [00:45<00:00,  9.10s/it]


   Model  Fold 1  Fold 2  Fold 3 Average        Time
0  Conv5  54.00%  48.58%  54.17%  52.25%  154.642134


Testing on Entire Test Set

In [ ]:
import torch.optim as optim
net = Conv4()

net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)  # Use the Adam optimizer



for epoch in tqdm(range(5)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

100%|██████████| 5/5 [00:48<00:00,  9.61s/it]

Finished Training


In [ ]:
net.eval()


correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)


        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 3000 test images: {100 * correct // total} %')

Accuracy of the network on the 3000 test images: 60 %


In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = net(inputs)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: cherry is 52.3 %
Accuracy for class: strawberry is 63.8 %
Accuracy for class: tomato is 67.1 %


Tune Epochs

In [ ]:
import pandas as pd
import torch.optim as optim
from sklearn.model_selection import KFold
import time
import torch.nn as nn
import torch.nn.functional as F

import warnings

# To ignore all warnings (not recommended unless you know the risks)
warnings.filterwarnings("ignore")


# Initialize an empty DataFrame to store the results
results = pd.DataFrame(columns=["Model", "Epochs", "Fold 1", "Fold 2", "Fold 3", "Average", "Time"])

# Define the list of epochs to test
epochs_list = [5, 10, 15]
batch_size = 64
folds = 3
learning_rate = 0.001
classes = ('cherry', 'strawberry', 'tomato')

# Create a KFold object to handle the splits for the training set
kf = KFold(n_splits=folds, shuffle=True)

for epochs in epochs_list:
    batch_scores = []
    batch_time = []
    for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset), 1):
        print(f"Testing Model Conv4, Epochs {epochs}, Fold {fold}:")

        train_split_dataset = torch.utils.data.Subset(train_dataset, train_indices)
        val_dataset = torch.utils.data.Subset(train_dataset, val_indices)

        trainloader = torch.utils.data.DataLoader(train_split_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
        valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

        net = Conv4()
        net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(net.parameters(), lr=learning_rate)

        start_time = time.time()
        for epoch in tqdm(range(epochs)):
            running_loss = 0.0
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        # Test the model on the separate test set
        net.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for data in valloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy_val = 100 * correct / total

        end_time = time.time()
        elapsed_time = end_time - start_time

        # Append fold score to batch_scores
        batch_scores.append(accuracy_val)
        # Append elapsed time to batch_time
        batch_time.append(elapsed_time)

    # Calculate the average score for the current number of epochs
    average_score = sum(batch_scores) / len(batch_scores)

    # Add the results to the DataFrame
    results = results.append({
        "Model": "Conv4",
        "Epochs": epochs,
        "Fold 1": f"{batch_scores[0]:.2f}%",
        "Fold 2": f"{batch_scores[1]:.2f}%",
        "Fold 3": f"{batch_scores[2]:.2f}%",
        "Average": f"{average_score:.2f}%",
        "Time": sum(batch_time)
    }, ignore_index=True)

# Display the DataFrame
print(results)


Testing Model Conv4, Epochs 5, Fold 1:


100%|██████████| 5/5 [00:54<00:00, 10.90s/it]


Testing Model Conv4, Epochs 5, Fold 2:


100%|██████████| 5/5 [00:49<00:00,  9.95s/it]


Testing Model Conv4, Epochs 5, Fold 3:


...

Testing Model Conv4, Epochs 15, Fold 2:


100%|██████████| 15/15 [02:24<00:00,  9.63s/it]


Testing Model Conv4, Epochs 15, Fold 3:


100%|██████████| 15/15 [02:22<00:00,  9.52s/it]


   Model Epochs  Fold 1  Fold 2  Fold 3 Average        Time
0  Conv4      5  63.67%  58.50%  61.08%  61.08%  164.105112
1  Conv4     10  71.00%  63.75%  65.17%  66.64%  299.549843
2  Conv4     15  56.50%  67.33%  62.58%  62.14%  445.004826


Testing on entire training set

In [ ]:
import torch.optim as optim
net = Conv4()

net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)  # Use the Adam optimizer



for epoch in tqdm(range(10)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

100%|██████████| 10/10 [01:59<00:00, 11.93s/it]

Finished Training


In [ ]:
net.eval()


correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)


        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 3000 test images: {100 * correct // total} %')

Accuracy of the network on the 3000 test images: 58 %


In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = net(inputs)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: cherry is 40.0 %
Accuracy for class: strawberry is 86.5 %
Accuracy for class: tomato is 49.7 %


Augmenting the Dataset with Horizontal Flip

In [ ]:
import os
import random
import shutil

# Define your original dataset directory and destination directories
original_dataset_dir = '/content/drive/MyDrive/traindata'
train_dir = '/content/drive/MyDrive/train'
test_dir = '/content/drive/MyDrive/test'


# Define the split ratio (e.g., 80% for training, 20% for testing)
split_ratio = 0.8

# Create destination directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Iterate through the subdirectories in your original dataset
for class_dir in os.listdir(original_dataset_dir):
    class_path = os.path.join(original_dataset_dir, class_dir)

    # Create subdirectories in the train and test directories
    train_class_dir = os.path.join(train_dir, class_dir)
    test_class_dir = os.path.join(test_dir, class_dir)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # List all the files in the class directory
    files = os.listdir(class_path)

    # Randomly shuffle the list of files
    random.shuffle(files)

    # Determine the split point based on the split_ratio
    split_point = int(len(files) * split_ratio)

    # Copy or move files to the train and test directories
    for i, file in enumerate(files):
        src = os.path.join(class_path, file)
        if i < split_point:
            dst = os.path.join(train_class_dir, file)
        else:
            dst = os.path.join(test_class_dir, file)
        shutil.copy(src, dst)  # Use shutil.move if you want to move the files

print("Data split and stored into training and test directories.")





Data split and stored into training and test directories.


In [ ]:
from torch.utils.data import Subset

# Define the root directory of your dataset and your desired image size
train_root = '/content/drive/MyDrive/train'
test_root = '/content/drive/MyDrive/test'
desired_size = (300, 300)  # Adjust to your desired size

# Create a dataset with the same transformations you are using
train_transform = transforms.Compose([
    transforms.Resize(desired_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomHorizontalFlip(p=0.5)# Resize images to the desired size
])

test_transform = transforms.Compose([
    transforms.Resize(desired_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))# Resize images to the desired size
])

train_dataset = ImageFolder(root=train_root, transform=train_transform)
test_dataset = ImageFolder(root=test_root, transform=test_transform)

batch_size = 64

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('cherry','strawberry','tomato')

In [ ]:
import torch.optim as optim
net = Conv4()

net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)  # Use the Adam optimizer



for epoch in tqdm(range(27)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

net.eval()


correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)


        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 3000 test images: {100 * correct // total} %')

100%|██████████| 27/27 [05:58<00:00, 13.26s/it]

Finished Training


Accuracy of the network on the 3000 test images: 77 %


In [ ]:
net.eval()


correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)


        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 3000 test images: {100 * correct // total} %')

Accuracy of the network on the 3000 test images: 77 %


In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = net(inputs)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: cherry is 80.0 %
Accuracy for class: strawberry is 79.7 %
Accuracy for class: tomato is 72.7 %


Random Rotation

In [ ]:
from torch.utils.data import Subset

# Define the root directory of your dataset and your desired image size
train_root = '/content/drive/MyDrive/train'
test_root = '/content/drive/MyDrive/test'
desired_size = (300, 300)  # Adjust to your desired size

# Create a dataset with the same transformations you are using
train_transform = transforms.Compose([
    transforms.Resize(desired_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees = (-30,30))# Resize images to the desired size
])

test_transform = transforms.Compose([
    transforms.Resize(desired_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))# Resize images to the desired size
])

train_dataset = ImageFolder(root=train_root, transform=train_transform)
test_dataset = ImageFolder(root=test_root, transform=test_transform)

batch_size = 64

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('cherry','strawberry','tomato')

In [ ]:
import torch.optim as optim
net = Conv4()

net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)  # Use the Adam optimizer



for epoch in tqdm(range(10)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

net.eval()


correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)


        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 3000 test images: {100 * correct // total} %')

100%|██████████| 10/10 [05:01<00:00, 30.19s/it]

Finished Training


Accuracy of the network on the 3000 test images: 63 %


Transfer Learning

In [ ]:
from torchvision.models import resnet18
from torch.optim import lr_scheduler

model = resnet18(weights='imagenet')
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs,3)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)

#step_lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=7, gamma=0.1)

for epoch in tqdm(range(20)):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

model.eval()


correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)


        # calculate outputs by running images through the network
        outputs = model(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 3000 test images: {100 * correct // total} %')

100%|██████████| 20/20 [05:37<00:00, 16.89s/it]

Finished Training


Accuracy of the network on the 3000 test images: 91 %


In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: cherry is 91.0 %
Accuracy for class: strawberry is 96.3 %
Accuracy for class: tomato is 89.0 %


In [ ]:
def train_show(network, data, targ, lossFunc, optimiser, epochs):
    lossHistory = []  # just to show a plot later...
    accuHistory = []

    for t in range(epochs):
        optimiser.zero_grad()      # Gradients accumulate by default, so don't forget to do this.

        y = network.forward(data)  # the forward pass

        loss = lossFunc(y,targ)    # recompute the loss
        loss.backward()            # runs autograd, to get the gradients needed by optimiser
        optimiser.step()           # take a step

        # just housekeeping and reporting
        accuracy = torch.mean((torch.argmax(y,dim=1) == targ).float())
        lossHistory.append(loss.detach().item())
        accuHistory.append(accuracy.detach())

    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.plot(lossHistory,'r'); plt.title("loss"); plt.xlabel("epochs")
    plt.subplot(1,2,2)
    plt.plot(accuHistory,'b'); plt.title("accuracy")